In [194]:
import os
import json
import numpy
import pandas
import ast
import re
import datetime
import pickle
import difflib

In [36]:
f = open("data/CrimeData/2011_01_assault_clustered.js", "r") 

In [37]:
lines = f.readlines()
f.close()

nHeaderLines = 3
nFooterLines = 3
records = len(lines) - nHeaderLines - nFooterLines


417

In [132]:
def parseLine(jsLine, crimeSet):
    cleanLine = jsLine.replace(" },\n", "")
    cleanLine = cleanLine.replace(" \"type\": \"Feature\", \"properties\": { \"Unnamed: 0\": ", "")
    cleanLine = cleanLine.split(",", 1)
    cleanLine = "{" + cleanLine[1]
    cleanLine = cleanLine.split(", \"geometry")
    cleanLine = cleanLine[0]
    cleanLine = re.sub("\"Offense 2\".*Location\"", "\"Location\"", cleanLine)
    #print(cleanLine)

    try:
        dictLine = ast.literal_eval(cleanLine)
        dictLine["DateTime"] = datetime.datetime.strptime(dictLine["Date"] + " " + dictLine["Time"], "%m\/%d\/%Y %I:%M %p")
        dictLine["CrimeSet"] = crimeSet
        return dictLine
    except ValueError:
        print("ERROR converting string")
        print(cleanLine)
        return "ERROR"


In [134]:
df = pandas.DataFrame()
path = "./data/CrimeData/"
allFiles = os.listdir(path)
nBadRecords = 0

for fileName in allFiles:
    print(fileName)
    f = open(path + fileName, "r") 
    lines = f.readlines()
    f.close()

    nHeaderLines = 3
    nFooterLines = 3
    crimeSet = fileName.split("_")[2]
    nRecords = len(lines) - nHeaderLines - nFooterLines
    print(nRecords)
    records = lines[nHeaderLines + 1:nHeaderLines + nRecords]
    iLine = nHeaderLines + 1
    for record in records:
        dictRecord = parseLine(record, crimeSet)
        if(dictRecord != "ERROR"):
            if(len(df.columns) == 0):
                df = pandas.DataFrame(dictRecord)
            else:
                df = df.append(dictRecord, ignore_index=True) 
        else:
            print("Error on line " + str(iLine))
            nBadRecords += 1;
        iLine += 1

2011_01_assault_clustered.js
411
2011_01_theft_clustered.js
713
2011_02_assault_clustered.js
375
2011_02_theft_clustered.js
603
2011_03_assault_clustered.js
466
2011_03_theft_clustered.js
935
2011_04_assault_clustered.js
438
2011_04_theft_clustered.js
942
2011_05_assault_clustered.js
483
2011_05_theft_clustered.js
922
2011_06_assault_clustered.js
467
2011_06_theft_clustered.js
876
2011_07_assault_clustered.js
423
2011_07_theft_clustered.js
953
2011_08_assault_clustered.js
410
2011_08_theft_clustered.js
805
2011_09_assault_clustered.js
357
2011_09_theft_clustered.js
848
2011_10_assault_clustered.js
364
2011_10_theft_clustered.js
960
2011_11_assault_clustered.js
382
2011_11_theft_clustered.js
840
2011_12_assault_clustered.js
391
2011_12_theft_clustered.js
800
2012_01_assault_clustered.js
438
2012_01_theft_clustered.js
697
2012_02_assault_clustered.js
413
2012_02_theft_clustered.js
680
2012_03_assault_clustered.js
464
2012_03_theft_clustered.js
816
2012_04_assault_clustered.js
458
2012_04

In [139]:
df.head()
pickleFile = open("df.pickle", "wb")
pickle.dump(df, pickleFile)
pickleFile.close()

In [243]:
df = pickle.load( open( "df.pickle", "rb" ))

In [245]:
df = df.drop(['Address,'], axis=1)
df.head()

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,g_clusterK9,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,6,2,1,0,0,0,1,4,0,3
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,6,2,1,0,0,0,1,4,0,3
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,3,2,0,0,0,0,1,4,1,3
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,6,5,1,2,0,0,1,4,0,6
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,6,5,1,2,0,0,1,4,0,6


In [146]:
## Get aldermanic districts
mprop = pandas.read_csv("./data/Mprop.csv")
mprop.columns

C:\Users\marie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,26,43,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['TAXKEY', 'SUB_ACCT', 'YR_ASSMT', 'CHK_DIGIT', 'TAX_RATE_CD',
       'PLAT_PAGE', 'HOUSE_NR_LO', 'HOUSE_NR_HI', 'HOUSE_NR_SFX', 'SDIR',
       'STREET', 'STTYPE', 'C_A_CLASS', 'C_A_SYMBOL', 'C_A_LAND', 'C_A_IMPRV',
       'C_A_TOTAL', 'C_A_EXM_TYPE', 'C_A_EXM_LAND', 'C_A_EXM_IMPRV',
       'C_A_EXM_TOTAL', 'P_A_CLASS', 'P_A_SYMBOL', 'P_A_LAND', 'P_A_IMPRV',
       'P_A_TOTAL', 'P_A_EXM_TYPE', 'P_A_EXM_LAND', 'P_A_EXM_IMPRV',
       'P_A_EXM_TOTAL', 'LAST_VALUE_CHG', 'REASON_FOR_CHG', 'CONVEY_DATE',
       'CONVEY_TYPE', 'CONVEY_FEE', 'CHG_NR', 'DIV_ORG', 'DIV_DROP',
       'OWNER_NAME_1', 'OWNER_NAME_2', 'OWNER_NAME_3', 'OWNER_MAIL_ADDR',
       'OWNER_CITY_STATE', 'OWNER_ZIP', 'LAST_NAME_CHG', 'NEIGHBORHOOD',
       'EXM_ACREAGE', 'EXM_PER_CT_LAND', 'EXM_PER_CT_IMPRV', 'BLDG_TYPE',
       'NR_STORIES', 'BASEMENT', 'ATTIC', 'NR_UNITS', 'BLDG_AREA', 'YR_BUILT',
       'SWIM_POOL', 'FIREPLACE', 'AIR_CONDITIONING', 'NR_ROOMS', 'BEDROOMS',
       'BATHS', 'POWDER_ROOMS', 'PARKING_SP

In [192]:
mprop["HOUSE_NR_SFX"] = mprop["HOUSE_NR_SFX"].fillna('')
mprop.loc[mprop["HOUSE_NR_SFX"].str.isalnum(), "HOUSE_NR_SFX"] = "-" + mprop["HOUSE_NR_SFX"]

mprop["FULL_ADDRESS"] = mprop["HOUSE_NR_LO"].map(str) + mprop["HOUSE_NR_SFX"] + " " + mprop["SDIR"] + " " + mprop["STREET"] + " " + mprop["STTYPE"]
mprop.head()

print(mprop['FULL_ADDRESS'].isnull().sum())
mprop['FULL_ADDRESS'] = mprop['FULL_ADDRESS'].fillna('')
print(mprop['FULL_ADDRESS'].isnull().sum())

,TAXKEY,SUB_ACCT,YR_ASSMT,CHK_DIGIT,TAX_RATE_CD,PLAT_PAGE,HOUSE_NR_LO,HOUSE_NR_HI,HOUSE_NR_SFX,SDIR,...,ELEMENT,MSLINK,MAPID,TAX_DELQ,BI_VIOL,RAZE_STATUS,MAP_NAME,MAP_EXT,Unnamed: 92,FULL_ADDRESS
0,19995111,0,2018,4,1,102,9202,9202,,N,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,9202 N 124TH ST
1,19996100,0,2018,4,1,102,12125,12125,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,12125 W COUNTY LINE RD
2,19996210,0,2018,8,1,104,12301,12301,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,12301 W COUNTY LINE RD
3,19997000,0,2018,3,1,101,11715,11715,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,11715 W COUNTY LINE RD
4,19998200,0,2018,1,1,101,11617,11617,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,11617 W COUNTY LINE RD


In [246]:
#df = df.drop(["Adermanic_District"], axis=1)
df["Aldermanic_District"] = numpy.nan
df.head()

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9,Aldermanic_District
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,NaN
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,NaN
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,2,0,0,0,0,1,4,1,3,NaN
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,5,1,2,0,0,1,4,0,6,NaN
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,5,1,2,0,0,1,4,0,6,NaN


In [ ]:
totalFound = 0
totalFuzzy = 0
totalNotFound = 0

for index, row in df.iterrows():
    if(index % 100 == 0):
        print("INDEX: " + str(index))
        print("found: " + str(totalFound))
        print("fuzzy: " + str(totalFuzzy))
        print("not found: " + str(totalNotFound))
        print("Null dists: " + str(df["Aldermanic_District"].isnull().sum()))
        pickleFile = open("alder.pickle", "wb")
        pickle.dump(df, pickleFile)
        pickleFile.close()
        
    rootAddr = row['Address'].split(" #")[0];
    found = mprop['FULL_ADDRESS'].str.contains(rootAddr).any()
    if found == True:
        totalFound += 1
        dist = mprop[mprop['FULL_ADDRESS']==rootAddr]['GEO_ALDER'].tolist()[0]
        df.loc[index, 'Aldermanic_District'] = dist
    else:
        print("    " + rootAddr)
        matches = difflib.get_close_matches(rootAddr, mprop['FULL_ADDRESS'].tolist(), 5)
        #mprop.loc[mprop["FULL_ADDRESS"] == matches, "GEO_ALDER"]
        dist = mprop[mprop["FULL_ADDRESS"].isin(matches)]["GEO_ALDER"].mode()
        #row['Aldermanic_District'] = dist[0]
        #df.iloc[index, 'Aldermanic_District'] = dist[0]
        df.loc[index, 'Aldermanic_District'] = dist[0]
        totalFuzzy += 1

INDEX: 0
found: 0
fuzzy: 0
not found: 0
Null dists: 71278
    5916 N 37TH ST
    5713 N 38TH ST
    4747 N 30TH ST
    4081 N 24TH ST
    5952 N 37TH ST
    4335 N 28TH ST
    6811 N TEUTONIA AV
    5450 N 36TH ST
    6921 N TEUTONIA AV
    5966 N 35TH ST
    3090 W VILLARD AV
    2024 W ATKINSON AV
    6955 N TEUTONIA AV
    2600 W RUBY AV
    3820 W GREEN TREE RD
    6909 N 37TH ST
    4870 N 22ND ST
    5814 N 76TH ST
    5461 N 64TH ST
    4051 N 69TH ST
    7705 W THURSTON AV
    5726 N 94TH ST
    7109 W SILVER SPRING DR
    4803-A N 66TH ST
    7115 W SILVER SPRING DR
    8546 W APPLETON AV
    7023 W SILVER SPRING DR
    8126 W MEDFORD AV
    5741 N 90TH ST
    3941 N 60TH ST
    5725 N 95TH ST
    1225 N WATER ST
    906 E CENTER ST
    1212 E OGDEN AV
    1400 E BRADY ST
    1406 N 22ND ST
    1013 N 10TH ST
INDEX: 100
found: 63
fuzzy: 37
not found: 0
Null dists: 71270
    3225 W WISCONSIN AV
    1444 N 17TH ST
    1536 N 35TH ST
    680 N 27TH ST
    750 N 24TH ST
    812 N 

In [228]:
a = "7061 N TEUTONIA AV #206"
poss = mprop['FULL_ADDRESS'].tolist();
matches = difflib.get_close_matches(a, poss, 5)
print(matches)
dist = mprop[mprop["FULL_ADDRESS"].isin(matches)]["GEO_ALDER"].mode()
dist[0]

['7061 N TEUTONIA AV', '7501 N TEUTONIA AV', '7026 N TEUTONIA AV', '7017 N TEUTONIA AV', '3706 N TEUTONIA AV']


1.0

In [287]:
df.head()

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9,Aldermanic_District
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,1.0
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,14.0
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,2,0,0,0,0,1,4,1,3,1.0
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,5,1,2,0,0,1,4,0,6,1.0
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,5,1,2,0,0,1,4,0,6,1.0
